In [1]:
# See https://www.tensorflow.org/versions/master/tutorials/mnist/pros/index.html

In [2]:
# MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting

 MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting

 MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [4]:
# Hyper-Parameters
batch_size = 50
epochs = 20000 * batch_size
learning_rate = 1e-4
keep_prob_training = 0.5
keep_prob_testing = 1.0

print_progress = 100

# Inputs and Variables
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
keep_prob = tf.placeholder("float")

In [5]:
# Parameter Initialization
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [6]:
# Model
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

## Layer 1
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

## Layer 2
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

## Fully connected layer
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## Readout Layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [7]:
# Cost
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

In [8]:
# Optimizer
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [9]:
# Run
import sys

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.initialize_all_variables())
for i in range(epochs / batch_size):
    xs, ys = mnist.train.next_batch(batch_size)
    if i % print_progress == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: xs, y_: ys, keep_prob: 1.0})
        print("step %d, training accuracy %g" % (i, train_accuracy))
        sys.stdout.flush()
    train_step.run(feed_dict={x: xs, y_: ys, keep_prob: keep_prob_training})

step 0, training accuracy 0.14


step 100, training accuracy 0.82


step 200, training accuracy 0.9


step 300, training accuracy 0.88


step 400, training accuracy 1


...

step 19500, training accuracy 1


step 19600, training accuracy 1


step 19700, training accuracy 1


step 19800, training accuracy 1


step 19900, training accuracy 1


In [ ]:
# Evaluate
print("test accuracy %g" % accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: keep_prob_testing}))